# Project 1: Write a data science blog post

1) Pick a dataset.

2) Pose at least three questions related to business or real-world applications of how the data could be used.

3) Create a Jupyter Notebook, using any associated packages you'd like, to:

Prepare data:

Gather necessary data to answer your questions
Handle categorical and missing data
Provide insight into the methods you chose and why you chose them
Analyze, Model, and Visualize

Provide a clear connection between your business questions and how the data answers them.

4) Communicate your business insights:

Create a Github repository to share your code and data wrangling/modeling techniques, with a technical audience in mind
Create a blog post to share your questions and insights with a non-technical audience

### Business Questions

1) What types of listings are popular?

2) When is the best time of the year to start a marketing campagn?

3) How well can we predict the price of a listing? What aspects correlate well to listing prices?

In [ ]:
# Import statements

import math
import json
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Load datasets

calendar_df = pd.read_csv('./data/calendar.csv')
listings_df = pd.read_csv('./data/seattle/listings.csv')
reviews_df = pd.read_csv('./data/seattle/reviews.csv')

### Explore *listings.csv*

In [ ]:
# Rename id column to listing_id to match with other dfs
listings_df = listings_df.rename(columns={'id':'listing_id'})

listings_df.head(20)

In [ ]:
# Check if there are duplicating rows for listing_ids
print("Max occurances for listing_id: ", listings_df['listing_id'].value_counts().max())

# Remove columns with more than 75% NaNs
drop_cols = listings_df.columns[listings_df.isnull().mean() > .75]
print(drop_cols)
listings_df = listings_df.drop(drop_cols, axis=1)

In [ ]:
listings_df.head()

### 1) What types of listings are popular?

In [ ]:
# Checking the counts for room_type
listings_df['room_type'].value_counts()

In [ ]:
# Separate dfs by room_type and remove rows without any reviews
listing_type_cols = ['neighbourhood_group_cleansed', 'property_type',
                     'accommodates', 'bathrooms', 'bedrooms', 'beds',
                     'bed_type', 'amenities', 'number_of_reviews']

entire_df = listings_df[(listings_df['room_type'] == 'Entire home/apt') & (listings_df['number_of_reviews'] > 0)][listing_type_cols]
private_df = listings_df[(listings_df['room_type'] == 'Private room') & (listings_df['number_of_reviews'] > 0)][listing_type_cols]
shared_df = listings_df[(listings_df['room_type'] == 'Shared room') & (listings_df['number_of_reviews'] > 0)][listing_type_cols]

In [ ]:
entire_df.head()

In [ ]:
# Sort dfs by number_of_reviews
entire_df.sort_values(by=['number_of_reviews'], ascending=False, inplace=True)
private_df.sort_values(by=['number_of_reviews'], ascending=False, inplace=True)
shared_df.sort_values(by=['number_of_reviews'], ascending=False, inplace=True)

In [ ]:
print('''Number of listings by accommodation type:\n Entire Place: {}\n Private Room: {}\n Shared Room: {}'''
      .format(len(entire_df), len(private_df), len(shared_df)))

### a. Plot heatmaps for popular and unpopular listings by neighbourhood and property type 
*Popularity is defined by the number of reviews present*


In [ ]:
# Entire Place

entire_20 = math.floor(len(entire_df)*.2)
popular_entire_df = entire_df.head(entire_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()
unpopular_entire_df = entire_df.tail(entire_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()

fig, ax =plt.subplots(1,2, figsize=(15,10))
fig.suptitle('Neighbourhoood by Property Type - Entire Place')

sns.heatmap(popular_entire_df,annot=True, fmt=".0f", ax=ax[0])
sns.heatmap(unpopular_entire_df,annot=True, fmt=".0f", ax=ax[1])

ax[0].set_title('Popular Listings')
ax[1].set_title('Unpopular Listings')

fig.show();

For listings with entire place rented, Apartments and Houses are frequent listings. However, the most popular properties appear to be Bed&Breakfast, Cabin, and Camper/RV located in following neighbourhoods: Rainier Valley, Queen Anne, Northgate,  and Ballard. Note that these types of properties are not available in every neighbourhoods. Among the most unpopular listings, Apartments and Houses had the lowest review counts.

In [ ]:
# Private Room

private_20 = math.floor(len(private_df)*.2)
popular_private_df = private_df.head(private_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()
unpopular_private_df = private_df.tail(private_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()

fig, ax =plt.subplots(1,2, figsize=(15,10))
fig.suptitle('Neighbourhoood by Property Type - Private Room')

sns.heatmap(popular_private_df,annot=True, fmt=".0f", ax=ax[0])
sns.heatmap(unpopular_private_df,annot=True, fmt=".0f", ax=ax[1])

ax[0].set_title('Popular Listings')
ax[1].set_title('Unpopular Listings')

fig.show();

For listings with private rooms, the most popular listings are located in Downtown and Queen Anne. In Downtown, Bed&Breakfasts and Lofts had the most rental activities, wheras Queen Anne had most reviews for Cabins and Camper/RVs. Among the least popular listings, still the apartments and houses had least average reviews. They are consistent across the neighbourhoods.

In [ ]:
# Shared Room

shared_20 = math.floor(len(shared_df)*.2)
popular_shared_df = shared_df.head(shared_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()
unpopular_shared_df = shared_df.tail(shared_20).groupby(['neighbourhood_group_cleansed', 'property_type']).number_of_reviews.mean().unstack()

fig, ax =plt.subplots(1,2, figsize=(15,5))
fig.suptitle('Neighbourhoood by Property Type - Shared Room')

sns.heatmap(popular_shared_df,annot=True, fmt=".0f", ax=ax[0])
sns.heatmap(unpopular_shared_df,annot=True, fmt=".0f", ax=ax[1])

ax[0].set_title('Popular Listings')
ax[1].set_title('Unpopular Listings')

fig.show();

For units with shared rooms, Apartments in University District was the most popular listing type.

### b. Observe accommodation details by popularity and check if there is any pattern
*Popularity is defined by the number of reviews present*


In [ ]:
# Entire Place
popular_accom_entire_df = entire_df.head(entire_20).groupby(['accommodates', 'bedrooms', 'beds', 'bed_type', 'bathrooms']).number_of_reviews.mean()
popular_accom_entire_df = popular_accom_entire_df.rename('average_review_count').reset_index()
popular_accom_entire_df.sort_values(by=['average_review_count'], ascending=False, inplace=True)

unpopular_accom_entire_df = entire_df.head(entire_20).groupby(['accommodates', 'bedrooms', 'beds', 'bed_type', 'bathrooms']).number_of_reviews.mean()
unpopular_accom_entire_df = unpopular_accom_entire_df.rename('average_review_count').reset_index()
unpopular_accom_entire_df.sort_values(by=['average_review_count'], ascending=True, inplace=True)

print("\nPopular Listings\n", popular_accom_entire_df.head(20))
print("\nUnpopular Listings\n", unpopular_accom_entire_df.head(20))

In [ ]:
# Private Room
popular_accom_private_df = private_df.head(private_20).groupby(['accommodates', 'bedrooms', 'beds', 'bed_type', 'bathrooms']).number_of_reviews.mean()
popular_accom_private_df = popular_accom_private_df.rename('average_review_count').reset_index()
popular_accom_private_df.sort_values(by=['average_review_count'], ascending=False, inplace=True)

unpopular_accom_private_df = private_df.head(private_20).groupby(['accommodates', 'bedrooms', 'beds', 'bed_type', 'bathrooms']).number_of_reviews.mean()
unpopular_accom_private_df = unpopular_accom_private_df.rename('average_review_count').reset_index()
unpopular_accom_private_df.sort_values(by=['average_review_count'], ascending=True, inplace=True)

print("\nPopular Listings\n", popular_accom_private_df.head(20))
print("\nUnpopular Listings\n", unpopular_accom_private_df.head(20))

When observed 20 most rated and 20 least rated listings (entire place & private room), I could not observe any patterns for both popular and unpopular listings. This suggests that accomodation details such as number of guests allowed, number of bedrooms/bathrooms, or bed type do not contribute to the popularity of listings.

### c. Check what kind of amenities are offered in popular listings 
*Popularity is defined by the number of reviews present*


In [ ]:
# Entire Place
popular_amen_entire_df = entire_df.head(20)[['amenities', 'number_of_reviews']]
popular_amen_entire_df.sort_values(by=['number_of_reviews'], ascending=False, inplace=True)

popular_amen_lst = popular_amen_entire_df['amenities'].to_list()
popular_amen_lst = ','.join(popular_amen_lst)
popular_amen_lst = popular_amen_lst.translate(str.maketrans({'{': '', '}': '', '"': ''})).strip().split(',')
popular_amen_lst = set(popular_amen_lst)


unpopular_amen_entire_df = entire_df.tail(20)[['amenities', 'number_of_reviews']]
unpopular_amen_entire_df.sort_values(by=['number_of_reviews'], ascending=True, inplace=True)

unpopular_amen_lst = unpopular_amen_entire_df['amenities'].to_list()
unpopular_amen_lst = ','.join(unpopular_amen_lst)
unpopular_amen_lst = unpopular_amen_lst.translate(str.maketrans({'{': '', '}': '', '"': ''})).strip().split(',')
unpopular_amen_lst = set(unpopular_amen_lst)

unique_popular_amen = popular_amen_lst - unpopular_amen_lst
unique_unpopular_amen = unpopular_amen_lst - popular_amen_lst

In [ ]:
print(unique_popular_amen) # What's available in popular listings are also available in unpopular listings 
print(unique_unpopular_amen)

In [ ]:
# Private Room
popular_amen_private_df = private_df.head(20)[['amenities', 'number_of_reviews']]
popular_amen_private_df.sort_values(by=['number_of_reviews'], ascending=False, inplace=True)

popular_amen_lst = popular_amen_private_df['amenities'].to_list()
popular_amen_lst = ','.join(popular_amen_lst)
popular_amen_lst = popular_amen_lst.translate(str.maketrans({'{': '', '}': '', '"': ''})).strip().split(',')
popular_amen_lst = set(popular_amen_lst)


unpopular_amen_private_df = private_df.tail(20)[['amenities', 'number_of_reviews']]
unpopular_amen_private_df.sort_values(by=['number_of_reviews'], ascending=True, inplace=True)

unpopular_amen_lst = popular_amen_private_df['amenities'].to_list()
unpopular_amen_lst = ','.join(unpopular_amen_lst)
unpopular_amen_lst = unpopular_amen_lst.translate(str.maketrans({'{': '', '}': '', '"': ''})).strip().split(',')
unpopular_amen_lst = set(unpopular_amen_lst)

unique_popular_amen = popular_amen_lst - unpopular_amen_lst
unique_unpopular_amen = unpopular_amen_lst - popular_amen_lst

print(unique_popular_amen)
print(unique_unpopular_amen)

According to my analysis, available amenities are not the factors of popular listings since everything available in popular listings are also available in unpopular listings

### d. Conclusion
According to my analysis, the popularity of a listing is defined by type and the location of the property. The trend differs by the room_type (ie. entire place, private room, and shared room). For listings with ehtire place, most popular listings were Bed&Breakfast, Cabin, and Camper/RV located in Rainier Valley, Queen Anne, Northgate, and Ballard. For listings with private rooms, Bed&Breakfast, Cabin in Downtown and Queen Anne were popular. For shared units, apartments in University District was the most popular listing type.

### 2) When is the best time of the year to start a marketing campagn?

In [ ]:
# Observe the data
calendar_df.head()

In [ ]:
calendar_df.info()

In [ ]:
# Convert string date to datetime
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

In [ ]:
# Remove columns with more than 75% NaNs
drop_cols = calendar_df.columns[calendar_df.isnull().mean() > .75]
print(drop_cols) # Did not need to remove any columns

In [ ]:
# Convert column values into boolean
bool_encoder = lambda x: 0 if x == 't' else 1 # not available=1, available=0
calendar_df['available'] = calendar_df['available'].map(bool_encoder)
calendar_df = calendar_df.rename(columns={'available':'not_available'})
calendar_df['not_available'].value_counts()

In [ ]:
# Aggregate calendar_df to extract occupancy_rate
num_listing_df = calendar_df[['date','listing_id']].groupby('date').count()
num_listing_df = num_listing_df.rename(columns={'listing_id':'num_listing'})

occupancy_df = calendar_df[['date', 'not_available']].groupby('date').mean()
occupancy_df = pd.concat([occupancy_df, num_listing_df], axis=1)
occupancy_df.head()

In [ ]:
# Plot occupancy rate by month to observe the pattern

plt.figure(figsize=(30,15))

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

plt.title('Occupancy Rate by Month')
plt.xlabel('Months')
plt.ylabel('Occupancy Rate')
plt.plot(occupancy_df.index, occupancy_df.not_available)
plt.show()

To answer Q2, I extracted the occupancy information by dates from *calendar.csv* and plotted a line graph. This is done by aggregating the data by dates and averaging the the counts of listings that are not available. This imposes an assumption that unavailable listings are already occupied by other users.

The overall trend shows declining occupancy rates. This could mean that more listings are being introduced as the time passes. However, we can see that it is not the case because the number of listings by month is fixed to 3818. Thus, we can translate that AirBNB renting activity in Seattle is decreasing.

There is a steep increase of occupancy in April 2016 which remains high until it shows another spike in July 2016 that starts to decline after Auguest 2016. This ties well with summar vacation time. If the company aims to start a marketing campaign in Seattle area, it should target users renting between April and Augest. There is another surge in January 2017, which could be contributed by the end-of-the-year holiday season.

### 3) How well can we predict the price of a listing? What aspects correlate well to listing prices?

In [ ]:
df_cols = ['neighbourhood_group_cleansed', 'property_type',
          'accommodates', 'bathrooms', 'bedrooms', 'beds','bed_type',
          'guests_included', 'requires_license', 'instant_bookable', 'review_scores_rating']

def clean_data(df, cols, target_col):
    # Fill numeric columns with the mean
    df = df[listings_df['price'].notnull()].reset_index()
    
    y = df[target_col]
    rm_sign = lambda x: float(x.translate(str.maketrans({'$': '', ',': ''})))
    y = y.apply(rm_sign)
    
    df = df[cols]
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)
    X = df
    
    return X, y

X, y = clean_data(listings_df, df_cols, 'price')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=100) 

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit

#Predict using your model
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)

#Score using your model
test_score = r2_score(y_test, y_test_preds)
train_score = r2_score(y_train, y_train_preds)

In [ ]:
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))